In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *

In [2]:
# bs=48
# bs=24
bs=128

In [3]:
torch.cuda.set_device(2)

RuntimeError: cuda runtime error (10) : invalid device ordinal at /opt/conda/conda-bld/pytorch_1544174967633/work/torch/csrc/cuda/Module.cpp:34

In [4]:
data_path = Config.data_path()

In [5]:
lang = 'nl'
# lang = 'zh'

In [6]:
name = f'{lang}wiki'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)
lm_fns = [f'{lang}_wt', f'{lang}_wt_vocab']

In [7]:
from nlputils import split_wiki,get_wiki

In [12]:
get_wiki(path,lang)

unzipping...
extracting...


In [8]:
!head -n4 {path}/{name}

<doc id="1" url="https://nl.wikipedia.org/wiki?curid=1" title="Albert Speer">
Albert Speer

Berthold Konrad Hermann Albert Speer (Mannheim, 19 maart 1905 – Londen, 1 september 1981) was een Duits architect en stedenbouwkundige. Tijdens de naziheerschappij over Duitsland (1933-1945) was hij vanaf 1937 rijksarchitect en vanaf 1942 minister van Bewapening. Door zijn vriendschap met Adolf Hitler en zijn ministerschap gold hij als een van de machtigste mannen van het Derde Rijk. Na de oorlog werd Speer in Neurenberg veroordeeld tot twintig jaar gevangenisstraf.


In [9]:
dest = split_wiki(path,lang)

/storage/nlwiki/docs already exists; not splitting


In [10]:
dest.ls()[:5]

[PosixPath('/storage/nlwiki/docs/Evangelie volgens Johannes.txt'),
 PosixPath('/storage/nlwiki/docs/Cricket.txt'),
 PosixPath('/storage/nlwiki/docs/Anna Achmanova.txt'),
 PosixPath('/storage/nlwiki/docs/Parachutespringen.txt'),
 PosixPath('/storage/nlwiki/docs/Lillo (Limburg).txt')]

In [11]:
data = (TextList.from_folder(dest)
            .split_by_rand_pct(0.1, seed=42)
            .label_for_lm()           
            .databunch(bs=bs, num_workers=1))

data.save(f'{lang}_databunch')
len(data.vocab.itos),len(data.train_ds)

(60000, 181546)

In [12]:
data.save(f'{lang}_databunch')

In [13]:
data = load_data(path, f'{lang}_databunch', bs=bs)

FileNotFoundError: [Errno 2] No such file or directory: '/storage/nlwiki/nl_databunch'

In [14]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.5, pretrained=False).to_fp16()

In [15]:
lr = 1e-2
lr *= bs/48  # Scale learning rate by batch size

In [16]:
learn.unfreeze()
learn.fit_one_cycle(1, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,3.484053,3.369243,0.375884,2:31:45


In [17]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)
learn.to_fp32().save(mdl_path/lm_fns[0], with_opt=False)
learn.data.vocab.save(mdl_path/(lm_fns[1] + '.pkl'))